# Montando o Google Drive

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


# Get Tweets

In [2]:
#This code creates the dataset from Corpus.csv which is downloadable from the
#internet well known dataset which is labeled manually by hand. But for the text
#of tweets you need to fetch them with their IDs.
import tweepy

# Twitter Developer keys here
# It is CENSORED

consumer_key = '8dQp98mqEXzyAytoF2bh1J2Kq'
consumer_key_secret = '4HIJqPGpFLS1CQjunxWub5P5RyEOCugOtj7rIb5Nzek3vhI3hU'
access_token = '1616194797521276942-R7rsFb0AOf5CrgFGsQ0xjZUwU07uGk'
access_token_secret = 'r8hKREq8PCHE5xDxUrgsDFpyWrP0zL08BRTLmvvS4HEa6'


auth = tweepy.OAuthHandler(consumer_key, consumer_key_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)



# This method creates the training set
def createTrainingSet(corpusFile, targetResultFile):
    import csv
    import time

    import pandas as pd

    df = pd.read_csv('/content/tash-pt.csv', dtype={'id_twitter': 'object', 'sentiment':'Int64'})

    counter = 0
    corpus = []

    for linha in df.itertuples():
      corpus.append({"tweet_id": linha.id_twitter, "label": linha.sentiment})

    sleepTime = 2
    trainingDataSet = []

    for tweet in corpus:
        try:
            tweetFetched = api.get_status(tweet["tweet_id"])
            print("Tweet fetched" + tweetFetched.text)
            tweet["text"] = tweetFetched.text
            trainingDataSet.append(tweet)
            time.sleep(sleepTime)

        except:
            print("Inside the exception - no:2")
            continue

    with open(targetResultFile, 'w') as csvfile:
        linewriter = csv.writer(csvfile, delimiter=',', quotechar="\"")
        for tweet in trainingDataSet:
            try:
                linewriter.writerow([tweet["tweet_id"], tweet["text"], tweet["label"], tweet["topic"]])
            except Exception as e:
                print(e)
    return trainingDataSet
"""
# Code starts here
# This is corpus dataset
corpusFile = "/content/tash-pt.csv"
# This is my target file
targetResultFile = "/content/targetResultFile.csv"
# Call the method
resultFile = createTrainingSet(corpusFile, targetResultFile)
"""

'\n# Code starts here\n# This is corpus dataset\ncorpusFile = "/content/tash-pt.csv"\n# This is my target file\ntargetResultFile = "/content/targetResultFile.csv"\n# Call the method\nresultFile = createTrainingSet(corpusFile, targetResultFile)\n'

# Pré-processamento

In [4]:
import glob
import os
import errno
import sys
import pandas as pd


import re
#import emoji
import unicodedata, string    # versao Laerte

from nltk import sent_tokenize


from sklearn.model_selection import train_test_split
from keras.utils import pad_sequences
import numpy as np

from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score  

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm


import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding


!git lfs install
!git clone https://huggingface.co/neuralmind/bert-base-portuguese-cased

!pip install transformers 
!pip install tokenizers
import numpy as np 
import pandas as pd 
import torch 
from torch.utils.data import Dataset, DataLoader 
from transformers import BertForSequenceClassification, BertTokenizer, pipeline, AdamW, get_linear_schedule_with_warmup
from sklearn import preprocessing
from tqdm import tqdm 
from tokenizers import BertWordPieceTokenizer




import seaborn as sns
import matplotlib.pyplot as plt

Error: Failed to call git rev-parse --git-dir: exit status 128 
Git LFS initialized.
Cloning into 'bert-base-portuguese-cased'...
remote: Enumerating objects: 39, done.
remote: Total 39 (delta 0), reused 0 (delta 0), pack-reused 39
Unpacking objects: 100% (39/39), 103.13 KiB | 6.45 MiB/s, done.
Filtering content: 100% (3/3), 1.30 GiB | 82.12 MiB/s, done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 92.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 29.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### classe: Buscape

In [5]:
class Buscape:
    
    def __init__(self, read_path):
        self.read_path = read_path
        
        #initializes the path of the archive containing the documents with reviews.
        #method used to read files in an folder.
        self.files = glob.glob(self.read_path)


        #initialize the path of the output file which will contain a normalized file with all reviews and thumbs Up and Down values.
        self.writefile_path = "/content/drive/MyDrive/FrameworkSentimento/Corpora/Reviews_corpus_buscape/annotated_out.txt"
        self.writefile_path2 = "/content/drive/MyDrive/FrameworkSentimento/Corpora/Reviews_corpus_buscape/out.txt"

        self.review_counter = 0
        self.pos_counter = 0
        self.neg_counter = 0


    #This method access file by file in a directory folder and writes an output containing
    #the users opinion, thumbsUp and thumbsDown values refering the reviews.
    def search_in_files(self, archive, writefile, writefile2):
        f_out_annot = open (writefile, "a", encoding="utf8")
        f_out = open (writefile2, "a", encoding="utf8")
        annot = []
        
        for name in archive:
            try:
                with open(name, encoding="utf8") as f_in: 
                    print(name, end = "\r")
                    if self.test_formatting(f_in): #If the file is not in the correct formatting, jumps to the next file.
                        recommends_value = self.searching_recommends_value(f_in)
                        if (recommends_value != "-1"  and recommends_value != "1"): #Tests if recommends value is 0 or 1, else ignores the file.
                            continue
                        opinion_str = self.searching_opinion(f_in)
                        if(opinion_str == ""):
                            continue  
                        self.review_counter = self.review_counter + 1
                        if(recommends_value == "-1"):
                            self.neg_counter = self.neg_counter + 1
                            annot.append(-1)
                        else:
                            self.pos_counter = self.pos_counter + 1
                            annot.append(1)
                    else:
                        continue

                #Writes in the output file the data collected from the file formatting it in a way to be used for next algorithms.
                #The formatting consists of the users review/opinion \t thumbsUp value \t thumbsDown value.
                #Separates a review from another by two end of lines (\n\n). 
                f_out_annot.write(opinion_str)
                f_out_annot.write("\t")
                f_out_annot.write(recommends_value)
                f_out_annot.write("\n\n\n")

                f_out.write(opinion_str)
                f_out.write("\n")
                f_out.write("MARKERENDOFREVIEW")
                f_out.write("\n\n\n")


            except IOError as exc:
                print("Erro no arquivo")
                if exc.errno != errno.EISDIR:
                    raise
            #print(name)
        f_out_annot.close()
        f_out.close()

        return annot

    #Checks line by line for the string "recommends" and return its value in the file
    def searching_recommends_value(self, file):
        line = file.readline()

        while ("recommends" not in line):
            if (len(line) != 0):
                line = file.readline()
            else:
                return

        if ("Yes" in line):
            recommends_value = "1"
        else:
            recommends_value = "-1"

        return recommends_value


    #This method search in the file for the text which have the reviewer opinion.
    #It returns a string with the opinion in the review
    def searching_opinion(self, file):
        str = file.read()
        str = str.split("<opinion>")
        str = str[1].split("</opinion>")
        str = str[0].replace("\n", " ")
        str = str.strip()
        return str


    #This method check if the first line is in the correct formatting.
    def test_formatting(self, file):
        line = file.readline()

        if ("xml version" not in line):
            #print("Arquivo", file.name, "fora de formatação\n")
            return False

        return True

    def norm_buscape(self):
        #Creates the output files clearing them, and closing.
        #This is used to not overlap data
        open(self.writefile_path, "w+", encoding="utf8").close()
        open(self.writefile_path2, "w+", encoding="utf8").close()
        
        annot = self.search_in_files(self.files, self.writefile_path, self.writefile_path2)

        print("CONCLUIDO Buscape:\n")
        print("Reviews: ", self.review_counter, "\n")
        print("Pos: ", self.pos_counter, "\n")
        print("Neg: ", self.neg_counter, "\n\n")

        return annot

### classe: TweetSent

In [6]:
class TweetSent:

    def __init__(self, read_path):
        self.read_path = read_path

        self.file_annotated_out = "/content/drive/MyDrive/FrameworkSentimento/Corpora/TweetSentBR/tweets/tweets_annotated_out.txt"
        self.file_out = "/content/drive/MyDrive/FrameworkSentimento/Corpora/TweetSentBR/tweets/tweets_out.txt"

    def extract_files(self):
        pass

    def norm_tweets(self):
        f1 = open(self.file_annotated_out, "w", encoding= "utf8")
        f2 = open(self.file_out, "w", encoding= "utf8")

        file_neg = self.read_path + "/tweets.neg"
        file_neu = self.read_path + "/tweets.neu"
        file_pos = self.read_path + "/tweets.pos"

        with open(file_neg, encoding="utf8") as f:
            for line in f:
                line = line[19:len(line)-1]
                f1.write(line + "\t-1\n\n\n")
                f2.write(line + "\nMARKERENDOFREVIEW\n\n\n")

        with open(file_neu, encoding="utf8") as f:
            for line in f:
                line = line[19:len(line)-1]
                f1.write(line + "\t0\n\n\n")
                f2.write(line + "\nMARKERENDOFREVIEW\n\n\n")

        with open(file_pos, encoding="utf8") as f:
            for line in f:
                line = line[19:len(line)-1]
                f1.write(line + "\t1\n\n\n")
                f2.write(line + "\nMARKERENDOFREVIEW\n\n\n")

        f1.close()
        f2.close()
        print("Concluído TweetSent\n\n")

### classe: FileToDataframe

In [7]:
class FileToDataframe:

    def __init__(self):
        pass
  
    # converte o arquivo para dataframe
    def ConvertToDataframe(self, dataset):
        try:
            df = pd.read_csv(dataset, sep="\t", header=None, names=['text','sent'])
            #df.text = df.text.astype(str)
            sent = [1,-1,0,"1","-1","0"]
          
            for i in range(len(df)):
                if df['sent'][i] in sent:
                    df.loc[i, 'sent'] = int(df['sent'][i])

            return df
        except IOerror as exc:
            print("Erro ao abrir arquivo")
            if exc.errno != errno.EISDIR:
                raise

### classe: Olist

In [8]:
df = FileToDataframe().ConvertToDataframe("/content/drive/MyDrive/FrameworkSentimento/Corpora/Reviews_corpus_buscape/annotated_out.txt")
df['sent'] = df['sent'].astype(object)
df['sent'] = df['sent'].replace(to_replace=[r"\n", r"\r"], value="", regex=True)

mascara = (df['sent'] == -1) | (df['sent'] == 1) 
df_filtrado = df.loc[mascara]

with open('olist_annotated_out_binary.txt', 'w', encoding='utf-8') as f:
   for index, row in df_filtrado.iterrows():
       linha = str(row['text']) + '\t' + str(row['sent']) + '\n\n\n'
       f.write(linha)

In [9]:
#path_dataset_annot = "/content/drive/MyDrive/FrameworkSentimento/Corpora/Reviews_corpus_buscape/annotated_out.txt"
#path_dataset_annot = "/content/drive/MyDrive/FrameworkSentimento/Corpora/TweetSentBR/tweets/tweets_annotated_out.txt"
#path_dataset_annot = "/content/drive/MyDrive/FrameworkSentimento/Corpora/tash_pt/tash_annotated_out.txt"
path_dataset_annot = "/content/drive/MyDrive/FrameworkSentimento/Corpora/Olist/olist_annotated_out_bkp.txt"
df = FileToDataframe().ConvertToDataframe(path_dataset_annot)
df['sent'].value_counts()

 1.0    26680
-1.0    11408
 0.0     3665
Name: sent, dtype: int64

### classe: tash-pt

In [10]:
df = pd.read_excel("/content/drive/MyDrive/FrameworkSentimento/Corpora/tash_pt/tash-pt.xlsx")
df['text'] = df['text'].replace(to_replace=[r"\n", r"\r"], value="", regex=True)

mascara = (df['label'] == -1) | (df['label'] == 1) 
df_filtrado = df.loc[mascara]

with open('tash_pt_out.txt', 'w', encoding='utf-8') as f:
   for index, row in df_filtrado.iterrows():
       linha = row['text'] + '\t'  + '\n\n\n'
       f.write(linha)

### classe: Normalize

In [11]:
class Normalize:

    def __init__(self):
        pass


    def buscape(self, dataset_path):
        buscape = Buscape(dataset_path)
        annot = buscape.norm_buscape()
        
        return annot
        

    def tweetsent(self, dataset_path):
        tweet = TweetSent(dataset_path)
        tweet.norm_tweets()

### classe: TwitterFilters

In [12]:
#!pip install emoji

import errno
import re
#import emoji
import unicodedata, string    # versao Laerte

class TwitterFilters:
    
    def __init__(self):
        self.path = "/content/drive/MyDrive/FrameworkSentimento/Corpora/TweetSentBR/tweets/tweets_out.txt"


    def __del_rt_infile(self):
        contents = ""
        try:
            f = open(self.path, "r", encoding="utf8")
            lines = f.readlines()
            for line in lines:
                if (line[0:3] == "RT "):
                    pos = lines.index(line)
                    lines.remove(line)
                    lines.pop(pos)
                    lines.pop(pos)
                    #lines.remove("\n")
                    #lines.remove("\n")
                    #print(lines)
            f.close()
        
        except IOError as exc:
            print("Erro no arquivo")
            if exc.errno != errno.EISDIR:
                raise

        try:
            f = open(self.path, "w+", encoding="utf8")
            f.writelines(lines)
            f.close()

        except IOError as exc:
            print("Erro no arquivo")
            if exc.errno != errno.EISDIR:
                raise

    def del_rt(self):
        self.__del_rt_infile()
        #self.__del_rt_infile(self.path_annot)

    def __del_words_tweetsent_infile(self):
        try:
            f = open(self.path, "r", encoding = "utf8")
            text = f.read()
            text = re.sub(r" ?NUMBER ?", " ", text)
            text = re.sub(r" ?USERNAME ?", " ", text)
            text = re.sub(r" ?http\S+ ?", " ", text)
            text = re.sub(r" ?@\S+ ?", " ", text)

            # filtros abaixo foram adicionados na versão do Laerte
            # para utilizar a versão do Felipe somente comentar os filtros abaixo
            #text = re.sub(r" ?#([^\s]+)", "", text)             # remove hashtag
            #text = emoji.get_emoji_regexp().sub("", text)       # remove emojis
            #text = re.sub("\d+|(\d*\.\d+)","", text)            # remove numeros
            #text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode("utf-8") # remove acentos das palavras
            #text = re.sub("\S*@\S*\s?","", text)    # remove email
            #text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text) # remove pontuacao
            #text = re.sub("\S+\.com\S*"," ", text)  # remove sites
            #text = re.sub("\s+"," ", text)          # remove espacos desnecessarios

            f.close()

        except IOerror as exc:
            print("Erro ao abrir arquivo")
            if exc.errno != errno.EISDIR:
                raise
        try:
            f = open(self.path, "w", encoding = "utf8")
            
            f.write(text)
            f.close()

        except IOerror as exc:
            print("Erro ao abrir arquivo")
            if exc.errno != errno.EISDIR:
                raise

    def del_words_tweetsent(self):
        self.__del_words_tweetsent_infile()
        #self.__del_words_tweetsent_infile(self.path_annot)

    def all_filters(self):
        self.del_rt()
        self.del_words_tweetsent()
        print("Filtros do Twitter Concluídos")

    def redo_abbrev(self):
        pass

### classe: PreparandoDataframeTweets

In [13]:
# arquivo do tweets chega sem os sentimentos, entao esse metodo coloca eles de volta
class PreparandoDataframeTweets:
    def __init__(self):
        pass

    def Atualizando(self, df):
        df2 = FileToDataframe().ConvertToDataframe("/content/drive/MyDrive/FrameworkSentimento/Corpora/TweetSentBR/tweets/tweets_out.txt")

        for i in range(len(df2)):
            if df2['text'][i] == "MARKERENDOFREVIEW":
                df2 = df2.drop(i,axis=0)

        df2.reset_index(drop=True, inplace=True)  # restaura os indices de cada linha

        for i in range(len(df)):
            df.loc[i, 'text'] = df2['text'][i]

### classe: BalancearDataframe

In [14]:
class BalancearDataframe:
    def __init__(self):
        pass


    def Balancear(self, dataframe):
        df = dataframe.copy()
        df.drop_duplicates(['text'], inplace=True) # remove textos duplicados
        df.reset_index(drop=True, inplace=True)

        quantPos = len(df[df['sent'] == 1])   # quantidade de positivos
        quantNeg = len(df[df['sent'] == -1])  # quantidade de negativos

        # faz a diferenca entre os dois e guarda o sentimento do maior
        if quantPos > quantNeg:
            diferenca = quantPos - quantNeg
            sent = 1
        else:
            diferenca = quantNeg - quantPos
            sent = -1
        
        # retira tweets do sentimento com maior quantidade
        cont = 1
        for i in range(len(df)):
            if cont <= diferenca:
                if df['sent'][i] == sent:
                    df = df.drop(i,axis=0)
                    cont+=1
        df.reset_index(drop=True, inplace=True)  # restaura os indices de cada linha

        return df


    def RemoverSentimento(self, dataframe, sent):
        df = dataframe.copy()
        df = df[df['sent'] != sent]
        df.reset_index(drop=True, inplace=True)  # restaura os indices de cada linha
        return df

### classe: ReviewsFilters

In [15]:
class ReviewsFilters:

    def __init__(self):
        pass


    # converte as palavras dos textos para minusculas
    def ConvertLowerCase(self, dataset):
        for i in range(len(dataset)):
            dataset.loc[i, 'text'] = dataset['text'][i].lower()
            #print(dataset['text'][i])


    # remove as letras repetidas das palavras
    def RemoveRepeatLetters(self, dataset):
        import re
        for i in range(len(dataset)):
            dataset.loc[i, 'text'] = re.compile(r"(.)\1{1,}", re.IGNORECASE).sub(r'\1', dataset['text'][i])
            #print(dataset['text'][i])


    # remove palavras desnecessarias
    def RemoveWords(self, dataset):
        import re, unicodedata, string
        for i in range(len(dataset)):
            dataset.loc[i, 'text'] = re.compile(r"&quot;|&amp", re.IGNORECASE).sub(r'', dataset['text'][i])  # remove "&quot;" e "&amp"
            dataset.loc[i, 'text'] = re.compile(r"r$", re.IGNORECASE).sub(r'', dataset['text'][i])      # remove r$
            dataset.loc[i, 'text'] = re.sub("\d+|(\d*\.\d+)","", dataset['text'][i])                    # remove numeros
            dataset.loc[i, 'text'] = unicodedata.normalize('NFKD', dataset['text'][i]).encode('ASCII', 'ignore').decode("utf-8") # remove acentos das palavras
            dataset.loc[i, 'text'] = re.sub("\S*@\S*\s?","", dataset['text'][i])  # remove email
            #dataset.loc[i, 'text'] = re.compile('[%s]' % re.escape(string.punctuation)).sub(' [%s]', dataset['text'][i]) # remove pontuacao
            dataset.loc[i, 'text'] = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', dataset['text'][i]) # remove pontuacao
            #dataset.loc[i, 'text'] = re.compile(r"\S,\S|\S\.\S|\S\:\S|\S\;\S\S\?\S\S\!\S\S\&\S", re.IGNORECASE).sub(r' ', dataset['text'][i]) # remove pontuacao no meio de palavras
            dataset.loc[i, 'text'] = re.sub("\S+\.com\S*"," ", dataset['text'][i])  # remove sites
            dataset.loc[i, 'text'] = re.sub("\s+"," ", dataset['text'][i])  # remove espacos desnecessarios


    def all_filters(self, dataset):
        df = dataset.copy()
        self.ConvertLowerCase(df)
        self.RemoveRepeatLetters(df)
        self.RemoveWords(df)

        return df

### classe: CorrecaoOrtografica

In [16]:
#!pip install symspellpy

#from symspellpy import SymSpell, Verbosity

class CorrecaoOrtografica:
  
    def __init__(self):
        self.sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
        self.sym_spell.load_dictionary("/content/drive/MyDrive/FrameworkSentimento/FrequenciaPalavras/pt_br_50k.txt", term_index=0, count_index=1)

    # corrige a ortografia de cada palavra contida no texto
    def corrigir(self, dataset):
        max_edit_distance = 2
        for i in range(len(dataset)):
            sugestoes = self.sym_spell.lookup_compound(dataset['text'][i], max_edit_distance)
    
            for sugestao in sugestoes:
                dataset.loc[i, 'text'] = sugestao.term
                print(dataset['text'][i])

### classe: RemocaoStopwords

In [17]:
class RemocaoStopwords:

    def __init__(self):
        pass


    # remove as stopwords utilizando a lista do NLTK
    def stopwordsNLTK(self, dataset):
        import nltk

        nltk.download('stopwords')
        nltk_stopwords = nltk.corpus.stopwords.words('portuguese')

        # remove palavras com sentido negativo
        nltk_stopwords = [i for i in nltk_stopwords if i not in ["não", "nenhum", "nada", "jamais", "nunca", "nem"]]

        for i in range(len(dataset)):
            word = dataset['text'][i].split(' ')
            listWords = [j for j in word if j not in nltk_stopwords]
            dataset.loc[i, 'text'] = ' '.join(listWords)
            #print(dataset['text'][i])


    # remove as stopwords utilizando a lista do Spacy
    def stopwordsSpacy(self, dataset):
        import spacy.cli, spacy

        spacy.cli.download("pt_core_news_sm")
        sp = spacy.load("pt_core_news_sm")
        spacy_stopwords = spacy.lang.pt.stop_words.STOP_WORDS

        # remove palavras com sentido negativo
        spacy_stopwords = [i for i in spacy_stopwords if i not in ["não", "nenhum", "nada", "jamais", "nunca", "nem"]]

        for i in range(len(dataset)):
            word = dataset['text'][i].split(' ')
            listWords = [j for j in word if j not in spacy_stopwords]
            dataset.loc[i, 'text'] = ' '.join(listWords)
            #print(dataset['text'][i])

### classe: Stemming

In [18]:
class Stemming:
  
    def __init__(self):
        pass
        

    # Reduz a palavra para o radical utilizando RSLP Stemmer
    def RSLP(self, dataset):
        import nltk

        nltk.download('rslp')
        stemmer = nltk.stem.RSLPStemmer()

        for i in range(len(dataset)):
            word = dataset['text'][i].split(' ')
            listWords = [stemmer.stem(j) for j in word if len(j) > 1]
            dataset.loc[i, 'text'] = ' '.join(listWords)
            #print(dataset['text'][i])


    # Reduz a palavra para o radical utilizando o Snowball
    def Snowball(self, dataset):
        import nltk

        stemmer = nltk.stem.SnowballStemmer("portuguese")

        for i in range(len(dataset)):
            word = dataset['text'][i].split(' ')
            listWords = [stemmer.stem(j) for j in word if len(j) > 1]
            dataset.loc[i, 'text'] = ' '.join(listWords)
            #print(dataset['text'][i])

### classe: SentenceSegment

In [19]:

from nltk import sent_tokenize
import errno



class SentenceSegment:
    def __init__(self):
       pass

    def tokenize(self, text):
        list = sent_tokenize(text)
        return list

### classe: Tokenizer

In [20]:
from nltk.tokenize import word_tokenize
import errno
from spacy.tokenizer import Tokenizer
from spacy.lang.pt import Portuguese
import re
        
sp = Portuguese()
tok = Tokenizer(sp.vocab)

#file_path = "/content/drive/MyDrive/FrameworkSentimento/Corpora/TweetSentBR/tweets/tweets_out.txt"

class Tokenizer:
    def __init__(self):
        pass


    def tokenize(self, text):

        list = word_tokenize(text)
        
        return list


    def tokenizeSpacy(self, text):
        tokens = str(tok(text))
        tokens = re.split(" |\n", tokens)
        
        if '' in tokens:
          tokens.remove('')

        return tokens

### classe: OpPhrasesDetector

In [21]:
import errno
import copy

class OpPhrasesDetector():
    
    def __init__(self):
        #self.document = document
        self.question_words_list = ["QUAL", "QUAIS", "QUE", "QUÊ", "ONDE", "QUANDO", "QUEM"]
        self.question_bigram = ["O", "POR"]
        self.question_mark = "?"

    def extract_text(self):
        try:
            f = open(self.document, "r", encoding="utf8")
            list = f.read().split("\n\n\n")
            f.close()

        except IOError as exc:
            print("Erro no arquivo")
            if exc.errno != errno.EISDIR:
                raise

        return list

    def write_text(self, path, words_list):
        open(path, "w").close()
        f = open (path, "a", encoding="utf8")
        for review in words_list:
            for sentence in review:
                for word in sentence:
                    f.write(word)
                    f.write(" ")
            f.write("\n\n\n")

        f.close()

    def sent_segment(self, text):
        review = SentenceSegment().tokenize(text)

        return review

    def sent_append(self, a_list):
        #a_list = self.extract_text()

        for review in a_list:
            a_list[a_list.index(review)] = self.sent_segment(review)

        return a_list

    def word_segment(self, sentence):
        a_list = Tokenizer().tokenize(sentence)

        return a_list

    def words_append(self, a_list):

        new_list = copy.deepcopy(a_list)

        for review in new_list:
            for sentence in review:
                review[review.index(sentence)] = self.word_segment(sentence)

        return new_list

    def check_question_bigram(self, a_list, a_str):
            if(a_list[a_list.index(a_str)-1] in self.question_bigram):
                return True
            else:
               return False

    def sentence_isinterrogation(self, sentence):
        for word in sentence:
            if(word.upper() in self.question_words_list):
                if((word.upper() == self.question_words_list[2]) or (word.upper() == self.question_words_list[3])):
                    if(self.check_question_bigram(sentence, word)):
                        pass
                    else:
                        continue

                if(self.question_mark == sentence[len(sentence)-1]):
                    return True
                else:
                    False

    def reconstruct_words_list(self, reviews_list):
        sentence_list = self.sent_append(reviews_list)
        words_list = self.words_append(sentence_list)
        for review in words_list:
            for sentence in review:
                if(self.sentence_isinterrogation(sentence)):
                    #print(review, "\n\n")
                    review.remove(sentence)
        return words_list

    def exec_opphrases(self, reviews_list, out_file):
        words_list = self.reconstruct_words_list(reviews_list)
        self.write_text(out_file, words_list)

### classe: POSTagger

In [22]:

import errno
import os


class POSTagger:

    def __init__(self):
        self.temp_path = "/content/drive/MyDrive/FrameworkSentimento/tempReview.txt"
        

    def open_temp_file(self, review):
        try:
            f = open(self.temp_path, "w+", encoding="utf8")
            f.write(review)
            f.close()

        except IOError as exc:
            print("Erro no arquivo")
            if exc.errno != errno.EISDIR:
                raise

    def freeling(self, review):
        self.open_temp_file(review)
        cmd = 'analyze -f pt.cfg < tempReview.txt > TreeTagger/cmd/tree-tagger-portuguese > saidaFreeling.txt'
        os.system(cmd)

    def freeling2(self):
        #self.open_temp_file(review)
        cmd = 'analyze -f pt.cfg < Reviews_corpus_buscape/out.txt > TreeTagger/cmd/tree-tagger-portuguese > saidaFreeling.txt'
        os.system(cmd)

    def linguakit(self, review):
        self.open_temp_file(review)
        cmd = '~/Linguakit-master/linguakit tagger pt tempReview.txt > TreeTagger/cmd/tree-tagger-portuguese > saidaLinguakit.txt'
        os.system(cmd)

    def treetagger(self, review):
        #self.open_temp_file(review)
        cmd = 'cat tempReview.txt | TreeTagger/cmd/tree-tagger-portuguese > saidaTTag.txt'
        os.system(cmd)

#pos_tag = POSTagger()
#pos_tag.freeling2()

# Técnicas de análise

### classe: LexLiu

In [23]:
import errno
import copy
import re

class LexLiu:
    def __init__(self):
        
        self.oplexicon_path = "/content/drive/MyDrive/FrameworkSentimento/OpLexicon/lexico_v3.0.txt"
        self.liwc_path = "/content/drive/MyDrive/FrameworkSentimento/LIWC/LIWC2007_Portugues_win.dic.txt"
        self.sentilex_path = "/content/drive/MyDrive/FrameworkSentimento/SentiLex-PT02/SentiLex-flex-PT02.txt"

        self.additives_conj_list = ["e"]
        self.adversative_conj_list = ["não", "nao"]##, "porém", "contudo", "todavia", "entretanto", "senão", "entanto", "obstante", "apesar"] comentar na escrita que para nivel de documento nao funciona tao bem
        self.intesifiers_list = []

        self.liwc = self.read_liwc()
        self.oplexicon = self.read_oplexicon()
        self.sentilex = self.read_sentilex()

    def read_liwc(self):

        formatted_liwc = []

        try:
            f = open(self.liwc_path, "r", encoding="utf8")
            liwc = f.readlines()
            f.close()
            for line in liwc:
                a_list = line.split()
                if("126" in a_list):
                    formatted_liwc.append([a_list[0], 1])
                elif("127" in a_list):
                    formatted_liwc.append([a_list[0], -1])
            
        except IOError as exc:
            print("Erro no arquivo")
            if exc.errno != errno.EISDIR:
                raise

        return formatted_liwc

    def read_oplexicon(self):
        formatted_oplexicon = []

        try:
            f = open(self.oplexicon_path, "r", encoding="utf8")
            oplexicon = f.readlines()
            f.close()

            for line in oplexicon:
                a_list = line.split(",")
                if("1" in a_list):
                    formatted_oplexicon.append([a_list[0], 1])
                elif("-1" in a_list):
                    formatted_oplexicon.append([a_list[0], -1])
            
        except IOError as exc:
            print("Erro no arquivo")
            if exc.errno != errno.EISDIR:
                raise

        return formatted_oplexicon

    def read_sentilex(self):
        formatted_sentilex = []

        try:
            f = open(self.sentilex_path, "r", encoding="utf8")
            sentilex = f.readlines()
            f.close()

            for line in sentilex:
                a_list = re.split(";|,|.PoS|=", line)
                if("POL:N1" in a_list):
                    pol = int(a_list[a_list.index("POL:N1")+1])
                    #str = str.split("=")
                    #pol = int(str[1])
                    formatted_sentilex.append([a_list[0], a_list[1], pol])
                else:
                    pol = int(a_list[a_list.index("POL:N0")+1])
                    #str = str.split("=")
                    #pol = int(str[1])
                    formatted_sentilex.append([a_list[0], a_list[1], pol])
                #elif("-1" in a_list):
                 #   formatted_oplexicon.append([a_list[0], "-1"])
            
        except IOError as exc:
            print("Erro no arquivo")
            if exc.errno != errno.EISDIR:
                raise

        return formatted_sentilex

    def search_liwc(self, word):

        polarity = 0

        for line in self.liwc:    
            if(word.lower() == line[0]):
                 polarity = line[1]
                 #print(line)
                        
                 return polarity

        return polarity

    def search_sentilex(self, word):
        
        polarity = 0

        for line in self.sentilex:    
            if(word.lower() == line[0]):
                 polarity = line[2]
                        
                 return polarity

            elif(word.lower() == line[1]):
                 polarity = line[2]
                        
                 return polarity

        return polarity

    def search_oplexicon(self, word):
        polarity = 0

        for line in self.oplexicon:    
            if(word.lower() == line[0]):
                 polarity = line[1]
                 #print(line)
                        
                 return polarity

        return polarity

    def annot_words_liwc(self, tokenized_review):
        nolexwords_t_review = copy.deepcopy(tokenized_review)
        annot_words_list = []
        conj = 0
        modifier = 1
        #polarity = 0
        
        #for word in tokenized_review:
        for i in range(0, len(tokenized_review)):  
            if(tokenized_review[i].lower() in self.adversative_conj_list):
                modifier = -1
                
                if(i < len(tokenized_review)-1):
                    continue

            elif(tokenized_review[i].lower() in self.intesifiers_list):
                elem = self.intesifiers_list[self.intesifiers_list.index(tokenized_review[i].lower())]
                modifier = elem[1]
                continue

            elif(tokenized_review[i].lower() in self.additives_conj_list):
                conj = 1
                
            else:
                annot = self.search_liwc(tokenized_review[i])
                if(annot != 0):
                    if(conj == 1):
                        if(len(annot_words_list)>0):
                            annot = annot_words_list[-1]
                            conj = 0                        
                    annot_words_list.append(modifier*annot)
                    nolexwords_t_review.remove(tokenized_review[i])
                    modifier = 1

            if(len(nolexwords_t_review) > 0):
                nolexwords_review = " ".join(nolexwords_t_review)
            else:
                nolexwords_review = " ".join(tokenized_review)
                
        return annot_words_list, nolexwords_review

    def annot_words_oplexicon(self, tokenized_review):
        nolexwords_t_review = copy.deepcopy(tokenized_review)
        annot_words_list = []
        conj = 0
        modifier = 1
        #polarity = 0
       
        #for word in tokenized_review:
        for i in range(0, len(tokenized_review)):
            if(tokenized_review[i].lower() in self.adversative_conj_list):
                modifier = -1
                
                if(i < len(tokenized_review)-1):
                    continue

            elif(tokenized_review[i].lower() in self.intesifiers_list):
                elem = self.intesifiers_list[self.intesifiers_list.index(tokenized_review[i].lower())]
                modifier = elem[1]
                continue

            elif(tokenized_review[i].lower() in self.additives_conj_list):
                conj = 1
                
            else:
                annot = self.search_oplexicon(tokenized_review[i])
                if(annot != 0):
                    if(conj == 1):
                        if(len(annot_words_list)>0):
                            annot = annot_words_list[-1]
                            conj = 0                        
                    annot_words_list.append(modifier*annot)
                    nolexwords_t_review.remove(tokenized_review[i])
                    modifier = 1

            if(len(nolexwords_t_review) > 0):
                nolexwords_review = " ".join(nolexwords_t_review)
            else:
                nolexwords_review = " ".join(tokenized_review)
        
        return annot_words_list, nolexwords_review

    def annot_words_sentilex(self, tokenized_review):
        nolexwords_t_review = copy.deepcopy(tokenized_review)
        annot_words_list = []
        conj = 0
        modifier = 1
        #polarity = 0
       
        #for word in tokenized_review:
        for i in range(0, len(tokenized_review)):
            if(tokenized_review[i].lower() in self.adversative_conj_list):
                modifier = -1
                
                if(i < len(tokenized_review)-1):
                    continue

            elif(tokenized_review[i].lower() in self.intesifiers_list):
                elem = self.intesifiers_list[self.intesifiers_list.index(tokenized_review[i].lower())]
                modifier = elem[1]
                continue

            elif(tokenized_review[i].lower() in self.additives_conj_list):
                conj = 1
                
            else:
                annot = self.search_sentilex(tokenized_review[i])
                if(annot != 0):
                    if(conj == 1):
                        if(len(annot_words_list)>0):
                            annot = annot_words_list[-1]
                            conj = 0                        
                    annot_words_list.append(modifier*annot)
                    nolexwords_t_review.remove(tokenized_review[i])
                    modifier = 1

            if(len(nolexwords_t_review) > 0):
                nolexwords_review = " ".join(nolexwords_t_review)
            else:
                nolexwords_review = " ".join(tokenized_review)
        
        return annot_words_list, nolexwords_review

### classe: Chi2

In [24]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import chi2
from sklearn.feature_extraction.text import CountVectorizer

class Chi2:
    
    def __init__(self):
        pass
        

    def x_to_dataframe(self, x_array):
        vect = CountVectorizer()
        x_dtm = vect.fit_transform(x_array)
        x_dtm = x_dtm.toarray()
        df = pd.DataFrame(x_dtm, columns = vect.get_feature_names_out())

        return df

    def calculate_chi2(self, df, y_array):
        chi2_score = chi2(df, y_array)

        return chi2_score

    def get_position_to_extract(self, chi2_score):
        position_to_extract = []
     
        for i in range(0, len(chi2_score[0])):
            if(chi2_score[0][i] >= 6.63):
                if(chi2_score[1][i] < 0.01):
                    position_to_extract.append(i)

        return position_to_extract

    def get_features(self, df, position_to_extract):
        extracted_features = []

        for pos in position_to_extract:
            extracted_features.append(df.columns[pos])

        return extracted_features

### classe: PMI

In [25]:
import pprint
import nltk
from math import log2
import errno

file_path = "/content/drive/MyDrive/FrameworkSentimento/Corpora/Reviews_corpus_buscape/out.txt"

class PMI:
    
    def __init__(self, document):
        self.document = document
        self.api_key = ""
        self.cse_id = ""
        #self.text = Tokenizer(self.document).tokenize()
        self.text = self.read_document()
        self.review_list = self.split_document()

    def read_document(self):
        try:
            f = open(self.document, "r", encoding="utf8")
            text = f.read()#.replace('\n', " ")
            f.close()

        except IOError as exc:
            print("Erro no arquivo")
            if exc.errno != errno.EISDIR:
                raise
        #text = text.split(" ")
        return text

    def split_document(self):
        a_list = self.text.split("\n\n\n")

        return a_list

    def pmi(self, phrase1, phrase2):
        p_phrase1 = self.pphrase(phrase1) + 0.01
        #print(phrase1,p_phrase1)
        p_phrase2 = 1#self.pphrase(phrase2) + 0.01
        #print(phrase2, p_phrase2)
        p_coocur = self.co_occur(phrase1, phrase2) + 0.01
        #print("co ocorencia: ",p_coocur)
        
        pmi_res = log2((p_coocur)/(p_phrase1*p_phrase2))
        
        return pmi_res

    def so(self, phrase):
        pmi_excelente = self.pmi(phrase, "excelente")
        #print(phrase," pmi_excelente: ",pmi_excelente, "\n")
        pmi_ruim = self.pmi(phrase, "ruim")
        #print(phrase, "pmi_ruim: ",pmi_ruim, "\n")
        so = pmi_excelente - pmi_ruim
        return so

    def pphrase(self, phrase):
        freq = self.text.upper().count(phrase.upper())
        return freq

    def co_occur(self, phrase1, phrase2):
        co_occur_freq = 0
        for review in self.review_list:
            if(phrase1.upper() in review.upper()):
                #print(review,"\n")
                if(phrase2.upper() in review.upper()):
                    #print(review,"\n\n")
                    co_occur_freq += 1
        return co_occur_freq

    def search_google(self, search_term, api_key, cse_id, **kwargs):
        service = build("customsearch", "v1", developerKey=api_key)
        res = service.cse().list(q=search_term, cx=cse_id, **kwargs).execute()
        return int(res['searchInformation']['totalResults'])

    def pmi_ir(self, phrase1, phrase2):
        query = phrase1 +" "+"AROUND(10)"+" "+ phrase2
        results = self.search_google(query, self.api_key, self.cse_id, num=10)
        return results

    def so_google(self, phrase):
        excelent_pmi = self.pmi_ir(phrase, "excelente")
        poor_pmi = self.pmi_ir(phrase, "péssimo")
        excelent_hits = self.search_google('excelente', self.api_key, self.cse_id, num=10)
        poor_hits = self.search_google('péssimo', self.api_key, self.cse_id, num=10)
        #print("Around Excelente: ", excelent_pmi)
        #print("Excelente: ", excelent_hits)
        #print("Around Péssimo: ", poor_pmi)
        #print("Péssimo: ", poor_hits)
        num = excelent_pmi*poor_hits
        den = poor_pmi*excelent_hits
        number = num/den
        sem_orientation = log2(number)
        return sem_orientation

buscape_out = "/content/drive/MyDrive/FrameworkSentimento/Corpora/Reviews_corpus_buscape/out.txt"
pmi_obj = PMI(buscape_out)
pmi_obj.review_list

#print(PMI(file_path).pmi("excelente", "péssimo"))
#print(PMI(file_path).co_occur("o que gostei", "o que não gostei"))
#print(PMI(file_path).pphrase(""))
#print("SO: ", PMI(file_path).so("ótimo"), "\n\n")

['Sobre o V3 voce esquece tudo, só lembra do seu desing e do status que esse telefone dá, principalmente o black.  O que gostei: É o status e o desing em pessoa.  O que não gostei: A falta de algumas funções como a navegação pela tecnologia EDGE.\nMARKERENDOFREVIEW',
 'O motorola v150 é um bom produto para quem não tem muita paciência para ficar procurando o menu que precisa, além de bonito é prático e conta com visor colorido.  O que gostei: Design compacto e moderno, suas teclas possibilitam fácil navegação entre os menus  O que não gostei: Dificuldade para enviar mensagens de texto\nMARKERENDOFREVIEW',
 'O aparelho é excelente, tem ótima autonomia da bateria e é muito fácil de usar. A conexão por infravermelho facilita muito a troca de informações com o computador.  O que gostei: Fácil utilização, bateria duradoura, muitas opções como agenda, histórico de ligações.  O que não gostei: É um pouco pesado e um pouco grande para os padrões atuais\nMARKERENDOFREVIEW',
 'minha opinião é qu

###SVM

In [26]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import svm

class SVM:

    def __init__(self):
        #Create a svm Classifier
        self.clf = svm.SVC(kernel='linear') # Linear Kernel

    def x_to_dataframe(self, x_array):
        vect = CountVectorizer()
        x_dtm = vect.fit_transform(x_array)
        x_dtm = x_dtm.toarray()
        return x_dtm

        #Train the model using the training sets
    def train_svm_model(self, x_train, y_train):
        return self.clf.fit(x_train, y_train)

        #Uses the test dataset to classify data
    def test_svm_model(self, x_test, trained_model):
        y_pred = trained_model.predict(x_test)
        return y_pred

    def svm_2(self):
        #model = svm.LinearSVC(C=0.1)
        #model = svm.SVC(gamma=0.001, C=100)
        model = svm.SVC(decision_function_shape='ovr', kernel='linear', cache_size=1000, degree=3, tol=0.001)
        return model

### classe: Liu

In [27]:
import time
import copy
from sklearn import metrics
#from keras.preprocessing.text import Tokenizer
#from sklearn.metrics import confusion_matrix, classification_report

import nltk
nltk.download('punkt')
import torch 

device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu") 
                                                                                                              
print("Conferindo a unidade de processamento:", device)

"""
path_buscape_reviews = "/content/drive/MyDrive/FrameworkSentimento/Corpora/Reviews_corpus_buscape/out.txt"
path_buscape_annot = "/content/drive/MyDrive/FrameworkSentimento/Corpora/Reviews_corpus_buscape/annotated_out.txt"
path_buscape_nointerrogation = "/content/drive/MyDrive/FrameworkSentimento/Corpora/Reviews_corpus_buscape/out_nointerrogation.txt" 
path_tweets_reviews = "/content/drive/MyDrive/FrameworkSentimento/Corpora/TweetSentBR/tweets/tweets_out.txt"
path_tweets_annot = "/content/drive/MyDrive/FrameworkSentimento/Corpora/TweetSentBR/tweets/tweets_annotated_out.txt"
path_temp_review = "/content/drive/MyDrive/FrameworkSentimento/tempReview.txt"
path_test_buscape = "/content/drive/MyDrive/FrameworkSentimento/testReviews.txt"
path_tes_tweets = "/content/drive/MyDrive/FrameworkSentimento/testReviewsTweets.txt"

path_read_buscape = "/content/drive/MyDrive/FrameworkSentimento/Corpora/Reviews_corpus_buscape/Reviews_corpus_buscape/*/*/*.xml"
path_read_tweets = "/content/drive/MyDrive/FrameworkSentimento/Corpora/TweetSentBR/tweets"
"""

class Liu():

    def __init__(self ,path_dataset_annot, path_dataset_out ):
        self.normalize = Normalize()
        self.twitter_filters = TwitterFilters()
        #self.test_list_buscape = self.read_review(path_buscape_reviews)
        self.test_list_tweets = self.read_review(path_dataset_out)
        #self.annot_list_buscape = self.read_review(path_buscape_annot)
        self.annot_list_tweets = self.read_review(path_dataset_annot)
        
        #self.test_list_tweets = test_list_tweets
        #self.annot_list_tweets = annot_list_tweets
        print(len(self.test_list_tweets))
        print(len(self.annot_list_tweets))
        self.tokenizer = Tokenizer()
        #self.sentence_segment = SentenceSegment()
        #self.opphrase = OpPhrasesDetector()
        self.lex = LexLiu()
        self.chi2 = Chi2()
        self.svm = SVM()

    def read_review(self, path):
        try:
            f = open(path, "r", encoding="utf8")
            review_list = f.read().split("\n\n\n")
            f.close()
            empty_counter = 0
            for review in review_list:
                if(review == ""):
                    empty_counter += 1
            
            for i in range(0, empty_counter):
                review_list.remove("")

            #print(empty_counter)

        except IOError as exc:
            print("Erro no arquivo")
            if exc.errno != errno.EISDIR:
                raise

        return review_list

    def raw_review(self, path):
        try:
            f = open(path, "r", encoding="utf8")
            review_list = f.read()
            #print(review)
            f.close()

        except IOError as exc:
            print("Erro no arquivo")
            if exc.errno != errno.EISDIR:
                raise

        return review_list

    def tokenize_review_list(self, review_list):

        tokenized_review_list = []

        for review in review_list:
            #tokenized_review_list.append(self.tokenizer.tokenize(review))     # tokenizer NLTK
            tokenized_review_list.append(self.tokenizer.tokenizeSpacy(review))     # tokenizer spaCy
       
        return tokenized_review_list

    def annot_review_list_liwc(self, review_list):
        pol_list = []
        t_reviews_list = self.tokenize_review_list(review_list)
        nolex_words_reviews_list = []
        counter = 0
        avg = 0
        soma = 0
        for review in t_reviews_list:
            start = time.time()
            pol, nolex_words = self.lex.annot_words_liwc(review)
            pol_list.append(pol)
            nolex_words_reviews_list.append(nolex_words)
            end = time.time()
            counter += 1
            soma = soma + (end-start)
            avg = '%.3f'%(soma/counter)
            print("LIWC\t", counter, " Time: ", '%.3f'%(end-start), " Average Time: ", avg)
        
        return pol_list, nolex_words_reviews_list

    def annot_review_list_oplexicon(self, review_list):
        pol_list = []
        t_reviews_list = self.tokenize_review_list(review_list)
        nolex_words_reviews_list = []
        counter = 0
        avg = 0
        soma = 0
        for review in t_reviews_list:
           start = time.time()
           pol, nolex_words = self.lex.annot_words_oplexicon(review)
           pol_list.append(pol)
           nolex_words_reviews_list.append(nolex_words)
           end = time.time()
           counter += 1
           soma = soma + (end-start)
           avg = '%.3f'%(soma/counter)
           print("Oplexicon\t", counter, " Time: ", '%.3f'%(end-start), " Average Time: ", avg)

        return pol_list, nolex_words_reviews_list

    def annot_review_list_sentilex(self, review_list):
      pol_list = []
      t_reviews_list = self.tokenize_review_list(review_list)
      nolex_words_reviews_list = []
      counter = 0
      avg = 0
      soma = 0
      for review in t_reviews_list:
          start = time.time()
          pol, nolex_words = self.lex.annot_words_sentilex(review)
          pol_list.append(pol)
          nolex_words_reviews_list.append(nolex_words)
          end = time.time()
          counter += 1
          soma = soma + (end-start)
          avg = '%.3f'%(soma/counter)
          print("Sentilex\t", counter, " Time: ", '%.3f'%(end-start), " Average Time: ", avg)

      return pol_list, nolex_words_reviews_list

    def sum_polarities(self, pol_list):
        pol_total = 0
        for pol in pol_list:
            pol_total = pol_total + pol
        
        return pol_total

    def make_final_pol_list(self, pol_list):
        final_pol_list = []
        
        for item in pol_list:
            final_pol_list.append(self.sum_polarities(item))
            #print(final_pol_list)

        return final_pol_list

    def normalize_pol_list(self, final_pol_list):
        
        normalized_pol_list =[]

        for pol in final_pol_list:
            if(pol > 0):
                normalized_pol_list.append(1)
            elif(pol < 0):
                normalized_pol_list.append(-1)
            else:
                normalized_pol_list.append(0)

        return normalized_pol_list

    def complete_normalize(self, pol_list):

        final_pol_list = self.make_final_pol_list(pol_list)
        normalized_pol_list = self.normalize_pol_list(final_pol_list)

        return normalized_pol_list

    def get_bunch_annot(self, annot, size):
        bunch_annot = []
        print(size)
        print(len(annot))
        for x in range(0, size):

            bunch_annot.append(annot[x])

        return bunch_annot

    def get_pol_from_txt(self, review_list):
        pol_list = []

        for line in review_list:
            review_line_list = line.split("\t")
            
            if (len(review_line_list) > 0):
                if("0" == review_line_list[-1]):
                    pol_list.append(0)
                elif("1" == review_line_list[-1]):
                    pol_list.append(1)
                elif("-1"  == review_line_list[-1]):
                    pol_list.append(-1)
                else:
                    pol_list.append(0)
         
        return pol_list

    """def get_lex_accuracy(self, lex_annot, annot):
        if(len(lex_annot) == len(annot)):
            accur = metrics.accuracy_score(lex_annot, annot)
            return accur
        
        else:
            print("Error, different sizes for annotation lists")
            
            return"""

    def calculate_metrics(self, pol_list, annot_list): 
        normalized_pol_list = self.complete_normalize(pol_list)
        bunch_annot = self.get_bunch_annot(self.get_pol_from_txt(annot_list), len(normalized_pol_list))
        print(metrics.classification_report(normalized_pol_list, bunch_annot))
        
    def calculate_metrics_liwc(self, test_list, annot_list):
        start = time.time()
        pol_list, nolex_words_review_list = self.annot_review_list_liwc(test_list)
        end = time.time()
        print("ANNNOT\t Time: ", '%.3f'%(end-start))
       
        self.calculate_metrics(pol_list, annot_list)
        

    def calculate_metrics_oplexicon(self, test_list, annot_list):

        pol_list = self.annot_review_list_oplexicon(test_list)
        self.calculate_metrics(pol_list, annot_list)

    def calculate_metrics_sentilex(self):
        
        pol_list = self.annot_review_list_sentilex(test_list)
        self.calculate_metrics(pol_list, annot_list)

    def remove_lexwords_reviews(self, review_list, lexicon):
        nolexwords_review_list = self.tokenize_review_list(review_list)
        s1 = []
        #print(nolexwords_review_list)
        for word in lexicon:
            #print(word[0])
            for review in nolexwords_review_list:
                for token in review:
                    if(word[0] == token.lower()):
                        review.remove(token)
            #print(word[0])
        for review in nolexwords_review_list:
                s1.append(" ".join(review))

        
        return s1

    def extract_neutral_reviews(self, pol_list, review_list):
        extracted_neu_reviews_list = []
        indexes_to_remove = []
        new_pol_list = copy.deepcopy(pol_list)
        new_review_list = copy.deepcopy(review_list)
        #new_annot_list = copy.deepcopy(annot_list)

        for i in range(0, len(pol_list)):
            if(pol_list[i] == 0):
                extracted_neu_reviews_list.append(review_list[i])
                indexes_to_remove.append(i)

        for index in indexes_to_remove:
            review_list[index] = "0"
            new_review_list[index] = "0"

        for pol in pol_list:
            if(pol == 0):
                new_pol_list.remove(pol)

        for review in review_list:
            if(review == "0"):
                new_review_list.remove(review)

        
        return extracted_neu_reviews_list, new_review_list, new_pol_list, indexes_to_remove

    def format_reviews_svm(self, reviews_dtm, indexes):
        test_reviews = []
        train_reviews = []
       # for index in indexes:
        #    extracted_dtm.append(reviews_dtm[index])

        for i in range(0, len(reviews_dtm)):
            for index in indexes:
                if(i == index):
                    test_reviews.append(reviews_dtm[index])
                    break
            else:
                train_reviews.append(reviews_dtm[index])


        return test_reviews, train_reviews

    def exec_chi2(self, pol_reviews, pol_list):
        df = self.chi2.x_to_dataframe(pol_reviews)
        chi2_score = self.chi2.calculate_chi2(df, pol_list)
        pos_to_extract = self.chi2.get_position_to_extract(chi2_score)
        opinion_indicators = self.chi2.get_features(df, pos_to_extract)

        return opinion_indicators

    def opinion_is_inreview(self, review, indicators):
        for ind in indicators:
            for word in review:
                if(ind == word):
                    return True

        return False

    def find_opinion_reviews(self, indicators, reviews):
        t_reviews = self.tokenize_review_list(reviews)
        indexes_exct_neu_reviews = []

        opinion_reviews = []

        for review in t_reviews:
            if(self.opinion_is_inreview(review, indicators)):
                opinion_reviews.append(" ".join(review))
                indexes_exct_neu_reviews.append(t_reviews.index(review))

        return opinion_reviews, indexes_exct_neu_reviews

    def translate_indexes(self, neu_indexes, exct_neu_indexes):
        new_indexes = []

        for index in exct_neu_indexes:
            new_indexes.append(neu_indexes[index])

        return new_indexes

    def select_test_reviews(self, test_reviews, exct_indexes):
        new_test_reviews = []

        for index in exct_indexes:
            new_test_reviews.append(test_reviews[index])

        return new_test_reviews

    def make_analysis_list(self, pol_list, to_append_pol):
        
        for item in to_append_pol:
            pol_list.append(item)

    def make_annot_pol_list(self, annot_reviews, size, indexes_to_remove, indexes_to_append):
        annot_pol_list = []

        pol_list = self.get_bunch_annot(self.get_pol_from_txt(annot_reviews), size)
        #pol_list = annot_reviews

        for i in range(0, len(pol_list)):
            if(i not in (indexes_to_remove)):
                annot_pol_list.append(pol_list[i])

        for index in indexes_to_append:
            annot_pol_list.append(pol_list[index])

        return annot_pol_list


"""

path_dataset_annot = "/content/drive/MyDrive/FrameworkSentimento/Corpora/Olist/olist_annotated_out_full_clean.txt"
path_dataset_out = "/content/drive/MyDrive/FrameworkSentimento/Corpora/Olist/olist_out_full_clean.txt"

#path_dataset_annot = "/content/drive/MyDrive/FrameworkSentimento/Corpora/TweetSentBR/tweets/tweets_annotated_out.txt"
#path_dataset_out = "/content/drive/MyDrive/FrameworkSentimento/Corpora/TweetSentBR/tweets/tweets_out.txt"


start_total = time.time()       
print('ok1')     
liu = Liu(path_dataset_annot, path_dataset_out)
#t_reviews_list = liu.tokenizer.tokenize(liu.test_list_tweets[1755])
#pol, nolex_words = liu.lex.annot_words_liwc(t_reviews_list)

#liu.twitter_filters.all_filters()
#print(liu.test_list_buscape)
#liu.normalize.tweetsent(path_read_tweets)
#annot = liu.normalize.buscape(path_read_buscape)
print('ok2')    

pol_list, nolex_words_review_list = liu.annot_review_list_sentilex(liu.test_list_tweets)

#print(len(nolex_words_review_list))
print('ok3')    
normalized_pol_list = liu.complete_normalize(pol_list)
print("Extracting neutral Reviews\n")
neu_reviews, pol_reviews, train_pol, neu_indexes = liu.extract_neutral_reviews(normalized_pol_list, nolex_words_review_list)
print("Finished extracting neutral reviews\n")


print("Executing Chi2\n")
opinion_indicators = liu.exec_chi2(pol_reviews, train_pol)
print("Finished Chi2\n")

print("Searching for extra opinative reviews\n")
extra_opinion_reviews, ind_exct_neu_reviews = liu.find_opinion_reviews(opinion_indicators, neu_reviews)
print("Finished search\n")

print("Creating DataFrame\n")
reviews_dtm = liu.svm.x_to_dataframe(nolex_words_review_list)
print('tam:' , len(neu_indexes))
test_reviews, train_reviews = liu.format_reviews_svm(reviews_dtm, neu_indexes)
print("Finished Dataframe\n")

print("Selecting reviews for test\n")
test_reviews = liu.select_test_reviews(test_reviews, ind_exct_neu_reviews)

print("Training model\n")
train_model = liu.svm.train_svm_model(train_reviews, train_pol)
print(type(test_reviews))
predict_list = liu.svm.test_svm_model(test_reviews, train_model)
print("Finished Training\n")

print("Making Analysis\n")
liu.make_analysis_list(train_pol, predict_list)
indexes_to_append = liu.translate_indexes(neu_indexes, ind_exct_neu_reviews)
annot_list = liu.make_annot_pol_list(liu.annot_list_tweets, len(normalized_pol_list), neu_indexes, indexes_to_append)
#print("Predict:\t", len(train_pol), "\nAnnot:\t", len(annot_list))
#print()
#print(metrics.classification_report(train_pol, annot_list))

#end_total = time.time()
#print("TOTAL TIME\t: ", '%.3f'%(end_total-start_total))
"""


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Conferindo a unidade de processamento: cuda:0


'\n\npath_dataset_annot = "/content/drive/MyDrive/FrameworkSentimento/Corpora/Olist/olist_annotated_out_full_clean.txt"\npath_dataset_out = "/content/drive/MyDrive/FrameworkSentimento/Corpora/Olist/olist_out_full_clean.txt"\n\n#path_dataset_annot = "/content/drive/MyDrive/FrameworkSentimento/Corpora/TweetSentBR/tweets/tweets_annotated_out.txt"\n#path_dataset_out = "/content/drive/MyDrive/FrameworkSentimento/Corpora/TweetSentBR/tweets/tweets_out.txt"\n\n\nstart_total = time.time()       \nprint(\'ok1\')     \nliu = Liu(path_dataset_annot, path_dataset_out)\n#t_reviews_list = liu.tokenizer.tokenize(liu.test_list_tweets[1755])\n#pol, nolex_words = liu.lex.annot_words_liwc(t_reviews_list)\n\n#liu.twitter_filters.all_filters()\n#print(liu.test_list_buscape)\n#liu.normalize.tweetsent(path_read_tweets)\n#annot = liu.normalize.buscape(path_read_buscape)\nprint(\'ok2\')    \n\npol_list, nolex_words_review_list = liu.annot_review_list_sentilex(liu.test_list_tweets)\n\n#print(len(nolex_words_revi

### classe: Turney

In [28]:
import errno
import time
from sklearn import metrics


from sklearn.metrics import accuracy_score  
from sklearn.preprocessing import MultiLabelBinarizer
import itertools
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix

class Turney():
    
    def __init__(self):
        #self.buscape_dataset_path = "/content/drive/MyDrive/FrameworkSentimento/Corpora/Reviews_corpus_buscape/Reviews_corpus_buscape/*/*/*.xml"
        #self.twitter_dataset_path = "/content/drive/MyDrive/FrameworkSentimento/Corpora/TweetSentBR/tweets"
        #self.tagger_temp_path = "/content/drive/MyDrive/FrameworkSentimento/tempReview.txt"
        self.treetagger_out = "/content/tash_out_tagg.txt"
        #self.freeling_out = "/content/drive/MyDrive/FrameworkSentimento/saidaFreeling.txt"
        #self.linguakit_out = "/content/drive/MyDrive/FrameworkSentimento/saidaLinguakit.txt"
        self.buscape_annot_out = "/content/drive/MyDrive/FrameworkSentimento/Corpora/tash_pt/tash_annotated_out.txt"
        self.buscape_out = "/content/drive/MyDrive/FrameworkSentimento/Corpora/tash_pt/tash_out.txt"
        #self.buscape_annot_out = "/content/drive/MyDrive/FrameworkSentimento/Corpora/Reviews_corpus_buscape/annotated_out.txt"
        #self.buscape_out = "/content/drive/MyDrive/FrameworkSentimento/Corpora/Reviews_corpus_buscape/out.txt"
        #self.buscape_out = self.buscape_out_aux[:10]
        #self.buscape_annot_out = self.buscape_annot_out_aux[:10]
        #self.buscape_annotations = "/mnt/d/Users/Felipe/Documents/Projects/Python Projects/Reviews_corpus_buscape/annotations.txt"
     

        #self.tagset = ["A", "C", "D", "N", "P", "R", "S", "V", "Z", "W", "I", "F"]
        self.adjective = "A"
        self.noun = "N"
        self.adverb = "R"
        self.verb = "V"

        self.normalize = Normalize()
        self.twitter_filters = TwitterFilters()
        self.tagger = POSTagger()
        self.PMI = PMI(self.buscape_out)

        self.review_list_aux = self.PMI.review_list
        self.review_list = self.review_list_aux
        self.buscape_annot_list_aux = self.get_buscape_annotations()
        self.buscape_annot_list = self.buscape_annot_list_aux
      
        
    
    def exec_norm_buscape(self):
        self.normalize.buscape(self.buscape_dataset_path)

    def exec_norm_twitter(self):
        self.normalize.tweetsent(self.twitter_dataset_path)

    def exec_twitter_filters(self):
        self.twitter_filters.all_filters()

    def list_tagged_words(self, path):
        words_list = []

        try:
            f = open(path, "r", encoding="utf8")
            lines = f.readlines()
            f.close()

        except IOError as exc:
            print("Erro no arquivo")
            if exc.errno != errno.EISDIR:
                raise

        for line in lines:
            words_list.append(line.split())
        #print(words_list)
        return words_list

    def find_pattern_FL(self, tagged_words):
        extracted_words_list = []

          
        for x in range(len(tagged_words)-1):
            #print(x)
            #print(tagged_words[x])
            if(tagged_words[x]==[]):
                continue
            if(tagged_words[x][2][0] == self.adjective):  #Tests if word is JJ
                if(x < (len(tagged_words)-1)):
                    if(tagged_words[x+1][2][0] == self.noun ):  #Tests if next word is NN
                        extracted_words_list.append(tagged_words[x][0] + " " + tagged_words[x+1][0]) #Extract Turney's first pattern JJ NN

                    elif(tagged_words[x+1][2][0] == self.adjective):
                        if(x < (len(tagged_words)-2)):
                            if(tagged_words[x+2][2][0] != self.noun ):
                                extracted_words_list.append(tagged_words[x][0] + " " + tagged_words[x+1][0]) #extract Turney's third pattern JJ JJ
                        elif(x == (len(tagged_words)-2)):
                            extracted_words_list.append(tagged_words[x][0] + " " + tagged_words[x+1][0])

            elif(tagged_words[x][2][0] == self.adverb):   #Tests if word is RB
                if(x < (len(tagged_words)-1)):
                    if(tagged_words[x+1][2][0] == self.verb ):  #Tests if next word is VB
                        extracted_words_list.append(tagged_words[x][0] + " " + tagged_words[x+1][0]) #Extract Turney's fifth pattern RB VB

                    elif(tagged_words[x+1][2][0] == self.adjective):
                        if(x < (len(tagged_words)-2)):
                            if(tagged_words[x+2][2][0] != self.noun ):
                                extracted_words_list.append(tagged_words[x][0] + " " + tagged_words[x+1][0]) #extract Turney's second pattern RB JJ
                        elif(x == (len(tagged_words)-2)):
                            extracted_words_list.append(tagged_words[x][0] + " " + tagged_words[x+1][0])
            

        #print(extracted_words_list)
        return extracted_words_list
    def find_pattern_TreeTagger(self, tagged_words):
        extracted_words_list = []
        
        for x in range(len(tagged_words)-1):
            if(tagged_words[x][1][0] == self.adjective):  #Tests if word is JJ
                if(x < (len(tagged_words)-1)):
                    if(tagged_words[x+1][1][0] == self.noun ):  #Tests if next word is NN
                        extracted_words_list.append(tagged_words[x][0] + " " + tagged_words[x+1][0]) #Extract Turney's first pattern JJ NN

                    elif(tagged_words[x+1][1][0] == self.adjective):
                        if(x < (len(tagged_words)-2)):
                            if(tagged_words[x+2][1][0] != self.noun ):
                                extracted_words_list.append(tagged_words[x][0] + " " + tagged_words[x+1][0]) #extract Turney's third pattern JJ JJ
                        elif(x == (len(tagged_words)-2)):
                            extracted_words_list.append(tagged_words[x][0] + " " + tagged_words[x+1][0])

            elif(tagged_words[x][1][0] == self.adverb):   #Tests if word is RB
                if(x < (len(tagged_words)-1)):
                    if(tagged_words[x+1][1][0] == self.verb ):  #Tests if next word is VB
                        extracted_words_list.append(tagged_words[x][0] + " " + tagged_words[x+1][0]) #Extract Turney's fifth pattern RB VB

                    elif(tagged_words[x+1][1][0] == self.adjective):
                        if(x < (len(tagged_words)-2)):
                            if(tagged_words[x+2][1][0] != self.noun ):
                                extracted_words_list.append(tagged_words[x][0] + " " + tagged_words[x+1][0]) #extract Turney's second pattern RB JJ
                        elif(x == (len(tagged_words)-2)):
                            extracted_words_list.append(tagged_words[x][0] + " " + tagged_words[x+1][0])
            
            elif(tagged_words[x][1][0] == self.noun):    #Tests if word is NN
                if(x < (len(tagged_words)-1)):
                    if(tagged_words[x+1][1][0] == self.adjective):   #Tests if nexto word is JJ
                            if(x < (len(tagged_words)-2)):
                                if(tagged_words[x+2][1][0] != self.noun ):
                                    extracted_words_list.append(tagged_words[x][0] + " " + tagged_words[x+1][0]) #extract Turney's third pattern NN JJ
                            elif(x == (len(tagged_words)-2)):
                                extracted_words_list.append(tagged_words[x][0] + " " + tagged_words[x+1][0])

        return extracted_words_list

    def calculate_review_so(self, extracted_words_list):
        review_so = 0
        for item in extracted_words_list:
            review_so += self.PMI.so(item)
            #print(review_so)
        if((len(extracted_words_list)) > 0):
            review_so = review_so/len(extracted_words_list)
       
        return review_so

    def gen_so_list(self, review_list):
        so_list =[]
        soma = 0
        x=0
        print(len(review_list))
        for x in range(len(review_list)-1):
          start = time.time() 
          self.tagger.freeling(review_list[x])
          tagged_words = self.list_tagged_words(self.treetagger_out)
          print(tagged_words)
          extracted_list = self.find_pattern_FL(tagged_words)
          so_list.append(self.calculate_review_so(extracted_list))
          end = time.time()
          soma = soma + (end-start)
          avg = '%.3f'%(soma/(x+1))
          print("TreeTagger\t", x, " Time: ", '%.3f'%(end-start), " Average Time: ", avg, "SO List: ", so_list[x])

        return so_list
    
    def get_buscape_annotations(self):
        pol_list = []
        try:
            f = open(self.buscape_annot_out, "r", encoding="utf8")
            review_list = f.read().split("\n\n\n")
            f.close()

            for line in review_list:
                review_line_list = line.split("\t")
            
                if (len(review_line_list) > 0):
                    if("0" == review_line_list[-1]):
                        pol_list.append(0)
                    elif("1" == review_line_list[-1]):
                        pol_list.append(1)
                    elif("-1"  == review_line_list[-1]):
                        pol_list.append(-1)

        except IOError as exc:
                print("Erro no arquivo")
                if exc.errno != errno.EISDIR:
                    raise

        return pol_list

    def normalize_pol_list(self, pol_list):
        
        normalized_pol_list =[]

        for pol in pol_list:
            if(pol > 0):
                normalized_pol_list.append(1)
            elif(pol < 0):
                normalized_pol_list.append(-1)
            else:
                normalized_pol_list.append(0)

        return normalized_pol_list
"""
start_total = time.time()        
turney = Turney()

#turney.exec_norm_twitter()
#turney.exec_twitter_filters()

so_list = turney.gen_so_list(turney.review_list)
normalized_so_list = turney.normalize_pol_list(so_list)
print(so_list)
print(normalized_so_list)
print(metrics.classification_report(normalized_so_list, turney.buscape_annot_list))

end_total = time.time()
print("TOTAL TIME\t: ", '%.3f'%(end_total-start_total))


print("Accuracia: ", accuracy_score(normalized_so_list, turney.buscape_annot_list))
print("Balanced Accuracy:", balanced_accuracy_score(normalized_so_list,  turney.buscape_annot_list))
print("F1 Score:", f1_score(normalized_so_list,  turney.buscape_annot_list, average='macro'))
print("Recall:", recall_score(normalized_so_list,  turney.buscape_annot_list, average='macro'))
print("Precision: ", precision_score(normalized_so_list,  turney.buscape_annot_list, average='macro'))

accuracia = accuracy_score(normalized_so_list, turney.buscape_annot_list)
balanced_accuracy = balanced_accuracy_score(normalized_so_list, turney.buscape_annot_list)
f1_Score = f1_score(normalized_so_list, turney.buscape_annot_list, average='macro')
recall =  recall_score(normalized_so_list, turney.buscape_annot_list, average='macro')
precision =  precision_score(normalized_so_list, turney.buscape_annot_list, average='macro')
"""

'\nstart_total = time.time()        \nturney = Turney()\n\n#turney.exec_norm_twitter()\n#turney.exec_twitter_filters()\n\nso_list = turney.gen_so_list(turney.review_list)\nnormalized_so_list = turney.normalize_pol_list(so_list)\nprint(so_list)\nprint(normalized_so_list)\nprint(metrics.classification_report(normalized_so_list, turney.buscape_annot_list))\n\nend_total = time.time()\nprint("TOTAL TIME\t: ", \'%.3f\'%(end_total-start_total))\n\n\nprint("Accuracia: ", accuracy_score(normalized_so_list, turney.buscape_annot_list))\nprint("Balanced Accuracy:", balanced_accuracy_score(normalized_so_list,  turney.buscape_annot_list))\nprint("F1 Score:", f1_score(normalized_so_list,  turney.buscape_annot_list, average=\'macro\'))\nprint("Recall:", recall_score(normalized_so_list,  turney.buscape_annot_list, average=\'macro\'))\nprint("Precision: ", precision_score(normalized_so_list,  turney.buscape_annot_list, average=\'macro\'))\n\naccuracia = accuracy_score(normalized_so_list, turney.buscape_

### classe: LSTM

In [30]:
from keras.layers.core import Activation, Dense, Dropout, SpatialDropout1D
#from keras.layers.embeddings import Embedding
from keras.layers import Embedding
#from keras.layers.recurrent import LSTM
from keras.layers import LSTM
from keras.layers import Bidirectional, Conv1D, MaxPooling1D, BatchNormalization
from keras.models import Sequential
from keras.regularizers import l2
#from keras.optimizers.schedules import ExponentialDecay não era utilizado?
from keras.optimizers import RMSprop

class LSTM_m:

    def __init__(self):
        pass


    def lstm_artigo(self, x, vocab_size):
        model = Sequential()
        model.add(Embedding(vocab_size,64, input_length=x.shape[1]))
        model.add(SpatialDropout1D((0.2)))
        model.add(LSTM(32, dropout=0.2, recurrent_dropout=0.2))
        model.add(Dense(2))
        model.add(Activation("sigmoid"))
        model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
        return model


    def lstm_artigo_vc(self, x, vocab_size):   # validacao cruzada
        def bm():
            model = LSTM_m().lstm_artigo(x, vocab_size)
            return model
        return bm

### classe: BERTIMBAU

In [32]:
"""
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding


!git lfs install
!git clone https://huggingface.co/neuralmind/bert-base-portuguese-cased

!pip install transformers 
!pip install tokenizers
import numpy as np 
import pandas as pd 
import torch 
from torch.utils.data import Dataset, DataLoader 
from transformers import BertForSequenceClassification, BertTokenizer, pipeline, AdamW, get_linear_schedule_with_warmup
from sklearn import preprocessing
from tqdm import tqdm 
from tokenizers import BertWordPieceTokenizer




import seaborn as sns
import matplotlib.pyplot as plt
"""

# Hiperparâmetros 
MAX_LENGTH = 512  
TRAIN_RATIO = 0.7
VAL_RATIO = 0.1
TEST_RATIO = 0.2
BATCH_SIZE = 32
EPOCHS = 4


device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu") 

print("Conferindo a unidade de processamento:", device)



class Bertimbau:

    def __init__(self):
        pass

    def cleaning_data(self, path_dataset):
      df = FileToDataframe().ConvertToDataframe(path_dataset)
      df = BalancearDataframe().Balancear(df) # Realiza o balanceamento do df 
      df.drop(df[df.sent == 0].index, inplace=True) # Retira os sentimentos neutros do df 
      df.drop_duplicates() # Remove as linhas duplicadas 
      mask = df['text'].apply(lambda x: len(x.split())) <= 500   # pro dataset "tash"
      df_filtrado = df[mask] # pro dataset "tash"

      return df_filtrado

    def size_of_sequences(self, df):
      token_lens = []
      max_length = 0


      for txt in df.text:
        tokens = txt.split()
        token_lens.append(len(tokens))
        if(len(tokens) > max_length ):
          max_length = len(tokens)

      sns.distplot(token_lens)
      plt.xlim([0, 300]);
      plt.xlabel('Token count')

      return max_length

    def tokenize(self, df, max_length):
      tokenizer = BertWordPieceTokenizer('/content/bert-base-portuguese-cased/vocab.txt', lowercase=False, strip_accents=False)
      print(max_length)
      if(max_length < 512):
        tokenizer.enable_truncation(max_length=max_length)
      else:
         tokenizer.enable_truncation(max_length=512)
      tokenizer.enable_padding()

      sentencas = df['text'].values
      labels = df['sent'].values
      output = tokenizer.encode_batch(sentencas.tolist())
      ids=[x.ids for x in output]
      attention_mask = [x.attention_mask for x in output]
      return ids, attention_mask

   

    
    def formatting_x_y(self, df, ids, attention_mask):
      X = torch.stack((torch.tensor(ids, dtype=torch.int), torch.tensor(attention_mask, dtype=torch.int)), dim=0)

      df['sent'] = df['sent'].apply(lambda x: 0 if x == -1 else 1)
      y = torch.Tensor(df['sent'].to_numpy())
      return df, X, y

    def split_data(self, dataset_len, dataset):

      # Responsável por instanciar nossos loaders de treino, validação e teste 

      num_train_instances = np.int(np.round(dataset_len * TRAIN_RATIO))
      num_val_instances = np.int(np.round(dataset_len * VAL_RATIO))
      num_test_instances = np.int(np.round(dataset_len * TEST_RATIO))
      print(f"Treino: {num_train_instances}, Val: {num_val_instances}, Teste: {num_test_instances}") 
      print('aqui')
      train_split, val_split, test_split = torch.utils.data.random_split(dataset, [num_train_instances, num_val_instances, num_test_instances]) 
      print('aqui 1')
      train_loader = torch.utils.data.DataLoader(train_split, batch_size=BATCH_SIZE, shuffle=True) # shuffle sorteia as instâncias a cada batch 
      val_loader = torch.utils.data.DataLoader(val_split, batch_size = BATCH_SIZE, shuffle=True)
      test_loader = torch.utils.data.DataLoader(test_split, batch_size=BATCH_SIZE, shuffle=True)
      print('aqui 2')
      return train_loader, val_loader, test_loader
    
    def model_version(self):
      config_model = int(input('Qual configuração do BERTIMBAU será usada? \nDigite 1 para usar o modelo BASE e 2 para usar o modelo LARGE '))
      if(config_model == 1):
        path = 'neuralmind/bert-base-portuguese-cased'
      elif(config_model == 2):
        path = 'neuralmind/bert-large-portuguese-cased'
      else:
        print('A configuração solicitada não existe')
      return path
    
    def setup_train(self, version, train_loader, val_loader, test_loader):
      epochs = 4
      steps_per_epoch = 200
      epoch_validation_samples = 50
      
      model = BertForSequenceClassification.from_pretrained(version,
      num_labels = 2, # NUMERO DE CLASSES (NO CASO BINÁRIA: ACEITÁVEL OU NÃO. PODE TER MAIS PARA MULTICLASSE)  
      output_attentions = True, # SE O MODELO DEVE EXPORTAR OS PESOS DAS ATENÇÕES
      output_hidden_states = True, # SE O MODELO DEVE EXPORTAR OS HIDDEN STATES (PODE SER INTERESSANTE PARA ESTUDAR EMBEDDINGS                                                     
      ).to(device) # DIZ AO MODELO PARA USAR GPU

      for param in model.base_model.parameters():
        param.requires_grad = True # Mantém os pesos do codificador pré-treinado congelados e otimiza apenas os pesos das camadas principais.

      
      loss_func = torch.nn.CrossEntropyLoss()

      optim = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )
      total_steps = epochs * len(train_loader) 

      scheduler = get_linear_schedule_with_warmup (optim, 
                                                  num_warmup_steps = 0, #VALOR PADRÃO
                                                  num_training_steps = total_steps)
      #return scheduler, model, optim

      list_train_acc = []
      list_val_acc = []
      list_train_loss = []
      list_val_loss = []
      epoch_metadata = []

      for i in range(epochs):
        num_train_examples = 0 
        num_val_examples = 0

        train_hits = 0
        val_hits = 0

        train_bar = tqdm(total=steps_per_epoch, desc=f"Train", unit="steps", position=0,leave=True)
        val_bar = tqdm(total=epoch_validation_samples, desc=f"Val", unit="samples",position=0,leave=True)

        for batch_number, (features, labels) in enumerate(train_loader):
          train_running_loss = 0
          model.train()
          input_ids, input_masks = features[:, 0, :], features[:, 1, :]

          #loss, logits = model(input_ids, input_masks, labels=labels.long())
          var_temp = model(input_ids, input_masks, labels=labels.long())

          loss, logits = var_temp[0], var_temp[1]

          
          optim.zero_grad()
          loss.backward()
          optim.step()

          train_running_loss += loss.item()

          softmax_predictions = torch.nn.functional.softmax(logits, dim=1)
          #train_hits += acc_calc(softmax_predictions, labels)


          train_bar.update()

          num_train_examples += features.shape[0]

          scheduler.step()

          if(batch_number + 1) % steps_per_epoch == 0:
            train_bar.close()
            break

        for batch_number, (features, labels) in enumerate(val_loader):
          with torch.no_grad():
            val_running_loss = 0

            model.eval()

            input_ids, input_masks = features[:, 0, :], features[:, 1, :]

          
            var_temp = model(input_ids, input_masks, labels=labels.long())

            loss, logits = var_temp[0], var_temp[1]

            val_running_loss += loss.item()

            softmax_predictions = torch.nn.functional.softmax(logits, dim=1)
            #val_hits += acc_calc(softmax_predictions, labels) 

            num_val_examples += features.shape[0]

            val_bar.update(1)


            if(batch_number + 1) % epoch_validation_samples ==0: 
              val_bar.close()
              break 

        train_acc = torch.true_divide(train_hits, num_train_examples) 
        val_acc = torch.true_divide(val_hits, num_val_examples) 
        list_train_loss.append(train_running_loss)
        list_val_loss.append(val_running_loss)
        list_train_acc.append(train_acc.item())
        list_val_acc.append(val_acc.item())


        #print(f"EPOCH SUMMARY- {i + 1} \t Train loss: { train_running_loss} \t Train Acc: {train_acc} \t Val loss: {val_running_loss} \t Val Acc: {val_acc}")

      num_test_examples = 0 


      resultados_esperados = []
      resultados_preditos = []
      texts = []

      train_hits = 0
      test_hits = 0 

      test_running_loss = 0 

      for batch_number, (features, labels) in enumerate(test_loader):
        with torch.no_grad():
          test_running_loss = 0 
          model.eval()

          input_ids, input_masks = features[:, 0, :], features[:, 1, :]

          #loss, logits = model(input_ids, input_masks, labels=labels.long())
          var_temp = model(input_ids, input_masks, labels=labels.long())

          loss, logits = var_temp[0], var_temp[1]

          test_running_loss += loss.item()

          softmax_predictions = torch.nn.functional.softmax(logits, dim=1)

          # Converte o Tensor de labels em List
          resultados_esperados.append(labels.cpu().numpy().tolist())

          # Converte o Tensor das predições em List
          resultados_preditos.append(softmax_predictions.argmax(axis = 1).cpu().numpy().tolist())  
        
        
          #test_hits += acc_calc(softmax_predictions, labels)
          num_test_examples += features.shape[0]



      # List of List to List
      resultados_esperados_1 = list(itertools.chain.from_iterable(resultados_esperados))
      resultados_preditos_1 = list(itertools.chain.from_iterable(resultados_preditos))


      accuracia = accuracy_score(resultados_esperados_1,resultados_preditos_1)
      balanced_accuracy = balanced_accuracy_score(resultados_esperados_1, resultados_preditos_1)
      f1_Score = f1_score(resultados_esperados_1, resultados_preditos_1, average='binary')
      recall =  recall_score(resultados_esperados_1, resultados_preditos_1, average='binary')
      precision =  precision_score(resultados_esperados_1, resultados_preditos_1, average='binary')

      print("Accuracia: ", accuracia)
      print("Balanced Accuracy:", balanced_accuracy)
      print("F1 Score:", f1_Score)
      print("Recall:", recall)
      print("Precision: ", precision)


      if(version == 'neuralmind/bert-base-portuguese-cased'):
        return ['BERTimbau base', accuracia, balanced_accuracy, f1_Score, recall, precision]
      elif(version == 'neuralmind/bert-large-portuguese-cased'):
        return ['BERTimbau large', accuracia, balanced_accuracy, f1_Score, recall, precision]
      

 
      

Conferindo a unidade de processamento: cuda:0


##classe: DataLoader

In [33]:
# Dataloader 
class TextDataset(Dataset):
  def __init__(self, X, y):
    self.X = X
    self.X = self.X.to(device)

    self.y = y
    self.y = self.y.to(device)

    self.len = len(y) # Calcula o tamanho do dataset 

  def __len__(self):
    return self.len   

  def __getitem__(self, idx):
    return self.X[:, idx], self.y[idx]

## classe: MetricasClassificacao

In [34]:
class MetricasClassificacao:

    def __init__(self):
        pass


    def treinoTeste(self, y_test, pred, algoritmo=None):
        if algoritmo == "lstm":
            import numpy as np
            y_test = np.argmax(y_test, axis=1)
            pred = np.argmax(pred, axis=1)

        #MetricasClassificacao().imprimindo(y_test, pred) 
        return y_test, pred


    def validacaoCruzada(self, model, x, y, folds, y_original=None, algoritmo=None):
      from sklearn.model_selection import cross_val_predict
      pred = cross_val_predict(model, x, y, cv=folds)
      print('algoritmo:', algoritmo)
      if algoritmo == "lstm":
        pred[pred==0] = -1
        y = y_original

      return y, pred


    def imprimindo(self, y, pred):
        import numpy as np
        from sklearn import metrics
        import matplotlib.pyplot as plt
        print("\n_____________________________________________________________")
        print("\t\tRelatorio de Classificação")
        print("-------------------------------------------------------------\n")
        print(metrics.classification_report(y, pred))
        print("_____________________________________________________________")
        print("\t\t  Matriz de Confusao")
        print("-------------------------------------------------------------\n")
        print(pd.crosstab(y, pred, rownames=['Real'], colnames=['Predito'], margins=True))

        print("\n_____________________________________________________________")
        print("\t\t\tCurva ROC")
        print("-------------------------------------------------------------\n")
        fpr, tpr, thresholds = metrics.roc_curve(y, pred)
        roc_auc = metrics.auc(fpr, tpr)

        plt.figure()
        plt.xlabel("Taxa de Falso Positivo")
        plt.ylabel("Taxa de Verdadeiro Positivo")
        plt.plot([0, 1], [0, 1], color="navy", linestyle="--")
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.title("Curva Roc")
        plt.plot(fpr, tpr, color="blue", lw=2, label='Area da curva roc = %0.2f)' % roc_auc)
        plt.legend(loc="lower right")
        plt.show()

        print("\n_____________________________________________________________\n")
        print(f"MAE: {metrics.mean_absolute_error(y, pred)}")

        mse = metrics.mean_squared_error(y, pred)
        print(f"MSE: {mse}")

        print(f"RMSE: {np.sqrt(mse)}")

        print(f"R_Square: {metrics.r2_score(y, pred)}")

        print("_____________________________________________________________\n")


# EXECUÇÃO

## Instancia os objetos

In [35]:
"""
from sklearn.model_selection import train_test_split
#from keras.preprocessing.sequence import pad_sequences
from keras.utils import pad_sequences
#from keras.preprocessing.text import Tokenizer
import numpy as np

from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score  

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
"""

def run_turney():
  start_total = time.time()        
  turney = Turney()

  #turney.exec_norm_twitter()
  #turney.exec_twitter_filters()

  so_list = turney.gen_so_list(turney.review_list)
  normalized_so_list = turney.normalize_pol_list(so_list)
  print(so_list)
  print(normalized_so_list)
  print(metrics.classification_report(normalized_so_list, turney.buscape_annot_list))

  end_total = time.time()
  print("TOTAL TIME\t: ", '%.3f'%(end_total-start_total))

  print("Accuracia: ", accuracy_score(normalized_so_list, turney.buscape_annot_list))
  print("Balanced Accuracy:", balanced_accuracy_score(normalized_so_list,  turney.buscape_annot_list))
  print("F1 Score:", f1_score(normalized_so_list,  turney.buscape_annot_list, average='macro'))
  print("Recall:", recall_score(normalized_so_list,  turney.buscape_annot_list, average='macro'))
  print("Precision: ", precision_score(normalized_so_list,  turney.buscape_annot_list, average='macro'))

  accuracia = accuracy_score(normalized_so_list, turney.buscape_annot_list)
  balanced_accuracy = balanced_accuracy_score(normalized_so_list, turney.buscape_annot_list)
  f1_Score = f1_score(normalized_so_list, turney.buscape_annot_list, average='macro')
  recall =  recall_score(normalized_so_list, turney.buscape_annot_list, average='macro')
  precision =  precision_score(normalized_so_list, turney.buscape_annot_list, average='macro')

  print("Accuracia: ", accuracia)
  print("Balanced Accuracy:", balanced_accuracy)
  print("F1 Score:", f1_Score)
  print("Recall:", recall)
  print("Precision: ", precision)

  return ['TURNEY', accuracia, balanced_accuracy, f1_Score, recall, precision]
  


def run_liu(path_dataset_annot, path_dataset_out):
 
  start_total = time.time()            
  liu = Liu(path_dataset_annot,path_dataset_out)
  #t_reviews_list = liu.tokenizer.tokenize(liu.test_list_tweets[1755])
  #pol, nolex_words = liu.lex.annot_words_liwc(t_reviews_list)

  #liu.twitter_filters.all_filters()
  #print(liu.test_list_buscape)
  #liu.normalize.tweetsent(path_read_tweets)
  #annot = liu.normalize.buscape(path_read_buscape)

  pol_list, nolex_words_review_list = liu.annot_review_list_sentilex(liu.test_list_tweets)
  #print(len(nolex_words_review_list))
  normalized_pol_list = liu.complete_normalize(pol_list)
  print("Extracting neutral Reviews\n")
  neu_reviews, pol_reviews, train_pol, neu_indexes = liu.extract_neutral_reviews(normalized_pol_list, nolex_words_review_list)
  print("Finished extracting neutral reviews\n")


  print("Executing Chi2\n")
  opinion_indicators = liu.exec_chi2(pol_reviews, train_pol)
  print("Finished Chi2\n")

  print("Searching for extra opinative reviews\n")
  extra_opinion_reviews, ind_exct_neu_reviews = liu.find_opinion_reviews(opinion_indicators, neu_reviews)
  print("Finished search\n")

  print("Creating DataFrame\n")
  reviews_dtm = liu.svm.x_to_dataframe(nolex_words_review_list)
  test_reviews, train_reviews = liu.format_reviews_svm(reviews_dtm, neu_indexes)
  print("Finished Dataframe\n")

  print("Selecting reviews for test\n")
  test_reviews = liu.select_test_reviews(test_reviews, ind_exct_neu_reviews)

  print("Training model\n")
  train_model = liu.svm.train_svm_model(train_reviews, train_pol)
  predict_list = liu.svm.test_svm_model(test_reviews, train_model)
  print("Finished Training\n")
  
  print("Making Analysis\n")
  liu.make_analysis_list(train_pol, predict_list)
  indexes_to_append = liu.translate_indexes(neu_indexes, ind_exct_neu_reviews)
  annot_list = liu.make_annot_pol_list(liu.annot_list_tweets, len(normalized_pol_list), neu_indexes, indexes_to_append)
  print("Predict:\t", len(train_pol), "\nAnnot:\t", len(annot_list))
  print()
  #print(metrics.classification_report(train_pol, annot_list))

  end_total = time.time()
  print("TOTAL TIME\t: ", '%.3f'%(end_total-start_total))

  accuracia = accuracy_score(train_pol,annot_list)
  balanced_accuracy = balanced_accuracy_score(train_pol, annot_list)
  f1_Score = f1_score(train_pol, annot_list, average='macro')
  recall =  recall_score(train_pol, annot_list, average='macro')
  precision =  precision_score(train_pol, annot_list, average='macro')

  print("Accuracia: ", accuracia)
  print("Balanced Accuracy:", balanced_accuracy)
  print("F1 Score:", f1_Score)
  print("Recall:", recall)
  print("Precision: ", precision)

  return ['LIU', accuracia, balanced_accuracy, f1_Score, recall, precision]
  


def run_bertimbau_model(path):
  bertimbau_object = Bertimbau()
  df = bertimbau_object.cleaning_data(path)
  
  max_length = bertimbau_object.size_of_sequences(df)
  ids, attention_mask = bertimbau_object.tokenize(df, max_length)
  
  df, X, y = bertimbau_object.formatting_x_y(df, ids, attention_mask )
  dataset = TextDataset(X, y)
  

  train_loader, val_loader, test_loader = bertimbau_object.split_data(dataset.len, dataset)
  version = bertimbau_object.model_version()
  print('opa')
  return bertimbau_object.setup_train(version, train_loader, val_loader, test_loader)

def run_svm_model(path):
  df = FileToDataframe().ConvertToDataframe(path)
  df.drop(df[df.sent == 0].index, inplace=True) # Retira os sentimentos neutros do df 
  df = BalancearDataframe().Balancear(df) 
  df = df.dropna(subset=['sent']) # Remove NaN
  df = df.drop(df[df['sent'] == '25 milímetros  O que não gostei: O tamanho do pixel'].index)
  #df['sent'] = df['sent'].astype(int)
  #classificador = SVM().x_to_dataframe(df['text'])
  #X_train, X_test, y_train, y_test = train_test_split(classificador, df['sent'], test_size=0.3, random_state=0)
  #model = svm.SVC(kernel='linear')    # modelo para a validacao cruzada
  #model = SVM().train_svm_model(X_train, y_train)
  #pred = SVM().test_svm_model(X_test, model)

  #Converte o texto em recursos de vetor
  vectorizer = TfidfVectorizer(min_df=0, max_df=0.9, sublinear_tf=True, use_idf=True, ngram_range=(1, 3)) 
  classificador = vectorizer.fit_transform(df['text'])

  # Divide o conjunto de dados em dados de treinamento e teste, com 30% dos dados para teste 
  X_train, X_test, y_train, y_test = train_test_split(classificador, df['sent'], test_size=0.3, random_state=0)

  model = SVM().svm_2()

  # Ajusta o SVM com os dados de treinamento
  model.fit(X_train, y_train)

  # O modelo faz as previsões nos dados de teste 
  pred = model.predict(X_test)


  accuracia = accuracy_score(y_test,pred)
  balanced_accuracy = balanced_accuracy_score(y_test, pred)
  f1_Score = f1_score(y_test, pred, average='binary')
  recall =  recall_score(y_test, pred, average='binary')
  precision =  precision_score(y_test, pred, average='binary')

  
  print("Accuracia: ", accuracia)
  print("Balanced Accuracy:", balanced_accuracy)
  print("F1 Score:", f1_Score)
  print("Recall:", recall)
  print("Precision: ", precision)
 
  return  ['SVM', accuracia, balanced_accuracy, f1_Score, recall, precision]
  

def run_lstm_model(path):
  from keras.preprocessing.text import Tokenizer
  
  df = FileToDataframe().ConvertToDataframe(path)
  df.drop(df[df.sent == 0].index, inplace=True) # Retira os sentimentos neutros do df 
  df = df.drop(df[df['sent'] == '25 milímetros  O que não gostei: O tamanho do pixel'].index)
  df = BalancearDataframe().Balancear(df) 
  tokenizer = Tokenizer(split=' ')
  tokenizer.fit_on_texts(df['text'].values)
  vocab_size = len(tokenizer.word_index) + 1

  X = tokenizer.texts_to_sequences(df['text'].values)
  Y = pd.get_dummies(df['sent']).values
  X = pad_sequences(X)


  # LSTM com treino e teste

  X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0)
  model = LSTM_m().lstm_artigo(X, vocab_size)
  model.fit(X_train, y_train, epochs = 2, batch_size=32)#, validation_split=0.1)
  pred = model.predict(X_test)

  y_test, pred = MetricasClassificacao().treinoTeste(y_test, pred, "lstm")


  accuracia = accuracy_score(y_test,pred)
  balanced_accuracy = balanced_accuracy_score(y_test, pred)
  f1_Score = f1_score(y_test, pred, average='binary')
  recall =  recall_score(y_test, pred, average='binary')
  precision =  precision_score(y_test, pred, average='binary')

  print("Accuracia: ", accuracia)
  print("Balanced Accuracy:", balanced_accuracy)
  print("F1 Score:", f1_Score)
  print("Recall:", recall)
  print("Precision: ", precision)

  print(df)
  return ['LSTM', accuracia, balanced_accuracy, f1_Score, recall, precision]

def run_lstm_model_vc(path):
  from keras.preprocessing.text import Tokenizer
  from keras.wrappers.scikit_learn import KerasClassifier
  
  df = FileToDataframe().ConvertToDataframe(path)
  df.drop(df[df.sent == 0].index, inplace=True) # Retira os sentimentos neutros do df 
  df = df.drop(df[df['sent'] == '25 milímetros  O que não gostei: O tamanho do pixel'].index)
  df = BalancearDataframe().Balancear(df) 
  tokenizer = Tokenizer(split=' ')
  tokenizer.fit_on_texts(df['text'].values)
  vocab_size = len(tokenizer.word_index) + 1

  X = tokenizer.texts_to_sequences(df['text'].values)
  Y = pd.get_dummies(df['sent']).values
  X = pad_sequences(X)


  # LSTM com treino e teste



  kmodel = KerasClassifier(build_fn=LSTM_m().lstm_artigo_vc(X, vocab_size), epochs = 5, batch_size=32)

  y_test, pred = MetricasClassificacao().validacaoCruzada(kmodel, X, Y, 2, df['sent'], "lstm")

  accuracia = accuracy_score(y_test,pred)
  balanced_accuracy = balanced_accuracy_score(y_test, pred)
  f1_Score = f1_score(y_test, pred, average='binary')
  recall =  recall_score(y_test, pred, average='binary')
  precision =  precision_score(y_test, pred, average='binary')

  print("Accuracia: ", accuracia)
  print("Balanced Accuracy:", balanced_accuracy)
  print("F1 Score:", f1_Score)
  print("Recall:", recall)
  print("Precision: ", precision)

  print(df)
  return ['LSTM Validação Cruzada', accuracia, balanced_accuracy, f1_Score, recall, precision]
  
  
"""
  X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0)
  model = LSTM_m().lstm_artigo(X, vocab_size)
  model.fit(X_train, y_train, epochs = 2, batch_size=32)#, validation_split=0.1)
  pred = model.predict(X_test)

  y_test, pred = MetricasClassificacao().treinoTeste(y_test, pred, "lstm")
"""

'\n  X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0)\n  model = LSTM_m().lstm_artigo(X, vocab_size)\n  model.fit(X_train, y_train, epochs = 2, batch_size=32)#, validation_split=0.1)\n  pred = model.predict(X_test)\n\n  y_test, pred = MetricasClassificacao().treinoTeste(y_test, pred, "lstm")\n'

## Run

In [36]:
df_metricas = pd.read_excel('/content/drive/MyDrive/FrameworkSentimento/Resultados.xlsx')
df_metricas

,Técnica,Dataset,Tempo Execução(seg),Accuracia,Balanced Accuracy,F1 Score,Recall,Precision
0,LIU,TweetSentBR,2442.737939,0.676282,0.679646,0.635434,0.679646,0.638167
1,LIU,Olist,8908.494408,0.686236,0.633364,0.370833,0.422243,0.378674
2,LIU,Buscapé,8061.894760,0.815934,0.548506,0.554136,0.548506,0.570197
3,LIU,tash_pt,217.496686,0.630584,0.630226,0.626204,0.630226,0.626827
4,BERTimbau base,TweetSentBR,285.142110,0.859003,0.859048,0.857633,0.862069,0.853242
5,BERTimbau base,Olist,268.612030,0.939726,0.940211,0.939086,0.912965,0.966746
6,BERTimbau base,Buscapé,97.115702,0.878187,0.878524,0.879552,0.902299,0.857923
7,BERTimbau base,tash_pt,18.615606,0.658824,0.661724,0.677778,0.743902,0.622449
8,BERTimbau large,TweetSentBR,946.069637,0.882786,0.882769,0.884422,0.883929,0.884916
9,BERTimbau large,Olist,848.452481,0.943607,0.943696,0.943569,0.952491,0.934812


In [ ]:
continuar = True
while continuar:
  print("O loop está sendo executado...")
  choose_dataset = int(input('Qual dataset será analisado? (digite o respectivo número do dataset)\n1.TweentSentBR\n2.Olist\n3.Buscapé\n4.tash\n'))
  analysis_technique = int(input('Qual técnica de análise? (digite o respectivo número da técnica)\n1.BERTimbau\n2.LSTM\n3.SVM\n4.Turney\n5.Liu\n6.LSTM Validação Cruzada\n'))

  if(choose_dataset == 1):
    path_dataset_annot = "/content/drive/MyDrive/FrameworkSentimento/Corpora/TweetSentBR/tweets/tweets_annotated_out_binary.txt"
    path_dataset_out = "/content/drive/MyDrive/FrameworkSentimento/Corpora/TweetSentBR/tweets/tweets_out_binary.txt"
  elif(choose_dataset == 2):
    path_dataset_annot = "/content/drive/MyDrive/FrameworkSentimento/Corpora/Olist/olist_annotated_out_bkp.txt"
    path_dataset_out = "/content/drive/MyDrive/FrameworkSentimento/Corpora/Olist/olist_out_bkp.txt"
  elif(choose_dataset == 3):
    path_dataset_annot = "/content/drive/MyDrive/FrameworkSentimento/Corpora/Reviews_corpus_buscape/annotated_out.txt"
    path_dataset_out = "/content/drive/MyDrive/FrameworkSentimento/Corpora/Reviews_corpus_buscape/out.txt"
  elif(choose_dataset == 4):
    path_dataset_annot = "/content/drive/MyDrive/FrameworkSentimento/Corpora/tash_pt/tash_annotated_out.txt"
    path_dataset_out = "/content/drive/MyDrive/FrameworkSentimento/Corpora/tash_pt/tash_out.txt"


  if (choose_dataset == 1):
    name_dateset = "TweetSentBR"
  elif(choose_dataset == 2):
    name_dateset = "Olist"
  elif(choose_dataset == 3):
    name_dateset = "Buscapé"
  elif(choose_dataset == 4):
    name_dateset = "tash_pt"


  if(analysis_technique == 1):
    inicio = time.time()
    resultados = run_bertimbau_model(path_dataset_annot)
    fim = time.time()
    tempo_execucao = fim - inicio
    resultados.insert(1, name_dateset)
    resultados.insert(2, tempo_execucao)
    df_metricas = df_metricas.append(pd.Series(resultados, index=df_metricas.columns), ignore_index=True)
  elif(analysis_technique == 2):
    inicio = time.time()
    resultados =run_lstm_model(path_dataset_annot)
    fim = time.time()
    tempo_execucao = fim - inicio
    resultados.insert(1, name_dateset)
    resultados.insert(2, tempo_execucao)
    df_metricas = df_metricas.append(pd.Series(resultados, index=df_metricas.columns), ignore_index=True)
  elif(analysis_technique == 3):
    inicio = time.time()
    resultados = run_svm_model(path_dataset_annot)
    fim = time.time()
    tempo_execucao = fim - inicio
    resultados.insert(1, name_dateset)
    resultados.insert(2, tempo_execucao)
    df_metricas = df_metricas.append(pd.Series(resultados, index=df_metricas.columns), ignore_index=True)
  elif(analysis_technique == 4):
    inicio = time.time()
    resultados = run_turney()
    fim = time.time()
    tempo_execucao = fim - inicio
    resultados.insert(1, name_dateset)
    resultados.insert(2, tempo_execucao)
    df_metricas = df_metricas.append(pd.Series(resultados, index=df_metricas.columns), ignore_index=True)
  elif(analysis_technique == 5):
    inicio = time.time()
    resultados = run_liu(path_dataset_annot, path_dataset_out)
    fim = time.time()
    tempo_execucao = fim - inicio
    resultados.insert(1, name_dateset)
    resultados.insert(2, tempo_execucao)
    df_metricas = df_metricas.append(pd.Series(resultados, index=df_metricas.columns), ignore_index=True)

  elif(analysis_technique == 6):
    inicio = time.time()
    resultados = run_lstm_model_vc(path_dataset_annot)
    fim = time.time()
    tempo_execucao = fim - inicio
    resultados.insert(1, name_dateset)
    resultados.insert(2, tempo_execucao)
    df_metricas = df_metricas.append(pd.Series(resultados, index=df_metricas.columns), ignore_index=True)

  
  resposta = input("Deseja continuar? (s/n)")
    # verifica se o usuário deseja sair do loop
  if resposta == "n":
    continuar = False


In [ ]:
df_metricas.to_excel('df_metricas.xlsx')